# Example 3 - Parameter Exploration of Spectral Analysis method

Due to the deterministic approach that [Tanaka *et al.*, 1999](http://linkinghub.elsevier.com/retrieve/pii/S0040195199000724) uses for estimating $Z_b$, the uncertainties returned by `CurieGrid.tanaka1999` are the simple results of error propagation from the initial spectra uncertainties. The spectra uncertainties represent the variance within the sampled sector of the 2D FFT, and may comprise both noise and anisotropic signal.

We present here several tests of the `CurieGrid.tanaka1999` function in order to explore how the input parameters affect the resulting Curie point depth estimates. In particular, we consider the following:

1. Effect of where in $k$-space is specific window used, separately for both the power and $k$-weighted power spectra (i.e., which $k$ do we consider?).
2. Effect of $k$-space window size (i.e., how much of the $k$-domain do we consider?). It is important to note that increasing the $k$-bandwidth also stabilises the estimation by increasing the statistics of the spectrum portion.
3. Effect of magnetic data window size (i.e., domain of magnetic data before computing FFT).

### Contents

- [Varying spatial frequency range](#Varying-spatial-frequency-range)
- [Varying spatial frequency size](#Varying-spatial-frequency-size)
- [Varying window size of magnetic anomaly](#Varying-window-size-of-magnetic-anomaly)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

In [ ]:
# load x,y,anomaly
mag_data = np.loadtxt("../../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

# initialise CurieOptimise object
grid = pycurious.CurieOptimise(d, xmin, xmax, ymin, ymax)

# pick centroid
xpt = xmin + (xmax-xmin)/2
ypt = ymin + (ymax-ymin)/2

## Varying spatial frequency range

This test explores where in $k$-space is specific window used, separately for both the power and $k$-weighted power.

In [ ]:
# 1) Where in k-space do we locate window?
window_size = 304e3
subgrid = grid.subgrid(window_size, xpt, ypt)
k, Phi, sigma_Phi = grid.radial_spectrum_log(subgrid, taper=None)

# Number of bins to divide k-range into
nbins = 10
hmap = np.zeros((nbins**2,3))

# k-range divided between 0 (i.e., DC), and 0.3 (i.e., spatial wavelengths of ~20 km)
kmin_range = np.linspace(0., 0.3, nbins)
kmax_range = kmin_range + (0.3/nbins)

for j in range(0,nbins):
    for l in range(0,nbins):
        kmin0 = kmin_range[j]
        kmin1 = kmax_range[j]
        kmax0 = kmin_range[l]
        kmax1 = kmax_range[l]
        
        (Ztr,btr,dZtr), (Zor, bor, dZor) = pycurious.tanaka1999(k, Phi, sigma_Phi, (kmin0, kmin1), (kmax0, kmax1))
        Zb,eZb = pycurious.ComputeTanaka(Ztr, dZtr, Zor, dZor)
        
        hmap[j*nbins+l,0] = (kmin0+kmin1)/2
        hmap[j*nbins+l,1] = (kmax0+kmax1)/2
        hmap[j*nbins+l,2] = Zb

In [ ]:
fig2 = plt.figure(figsize=(16,16))

ax2 = fig2.add_subplot(111)
ax2.set_xlabel(r'$\nu$ window of $\Phi/|k|$ (km$^{-1}$) (for Zo)')
ax2.set_ylabel(r'$\nu$ window of $\Phi$ (for Zt)')

# SCATTER PLOT
im2 = ax2.scatter(hmap[:,1], hmap[:,0], 2000, np.clip(hmap[:,2],10,20), marker='s')
fig2.colorbar(im2, label='CPD (km)')
plt.title('Heatmap of CPD')

Results of this test suggest that the precise $k$ location of $Z_t$ is significantly less influential than the $k$ location of $Z_o$. Furthermore, for this test case with a known $Z_b$ of 16 km, a spatial frequency $\nu$ window of $0.125-0.2$ (i.e., $k\in\{0.8-1.25\}$ gives estimates consistent with the true $Z_b$. For simplicity, based upon this test we see minimal reason to not use the same $\nu/k$ window for each spectrum.

## Varying spatial frequency size

Explore the effect of $k$-space window size (i.e., how much of the $k$-domain do we consider?). Increasing the $k$-bandwidth also stabilises the estimation by increasing the statistics of the spectrum portion.

In [ ]:
# 2) How big a bandwidth in k-space do we want?

k, Phi, sigma_Phi = grid.radial_spectrum_log(subgrid, taper=None)

# We are testing this by taking one central spatial frequency (k=1), and estimating Zb with increasing k-bandwidths.
# Recall k=2*pi*nu
nu = 1.0/(2.0*np.pi)

nbins = 30
output = np.zeros((nbins,3))
nr = np.linspace(0., 0.3, nbins+1)
output[:,0] = nr[1:]

for j in range(0,nbins,1):
    kmin = nu - 0.5*output[j,0]
    kmax = nu + 0.5*output[j,0]
    
    (Ztr,btr,dZtr), (Zor, bor, dZor) = pycurious.tanaka1999(k, Phi, sigma_Phi, (kmin, kmax), (kmin, kmax))
    Zb, eZb = pycurious.ComputeTanaka(Ztr, dZtr, Zor, dZor)
    output[j,1] = Zb
    output[j,2] = eZb
    
fig2 = plt.figure(figsize=(8,8))
ax2 = fig2.add_subplot(111)
ax2.errorbar(output[:,0], output[:,1], yerr=output[:,2])
ax2.invert_yaxis()
ax2.set_xlabel('$\\nu$-bandwidth')
ax2.set_ylabel('CPD depth (km)')

The CPD estimates resulting from this test become consistent for $\nu$-bandwidths greater than approximately 0.1. Statistics improve with larger bandwidths, however, it is worth remaining aware of the typically non-linear trend of the spectrum, as it may not be appropriate for a linear fit over a large domain of $\nu$.

## Varying window size

Effect of magnetic data window size - i.e., domain of magnetic data before computing the FFT.

In [ ]:
# 3) How large a window width of magnetic data is required?
grid = pycurious.CurieOptimise(d, xmin, xmax, ymin, ymax)
xpt = xmin + (xmax-xmin)/2
ypt = ymin + (ymax-ymin)/2

nwin = 50
baseW = 10000
inc = 2500

output = np.zeros((nwin,3))

# Define 'ws' as variable window size index
for i in range(0, nwin):
    ws = baseW + i*inc
    subgrid = grid.subgrid(ws, xpt, ypt)
    k, Phi, sigma_Phi = grid.radial_spectrum_log(subgrid, taper=None)
    (Ztr,btr,dZtr), (Zor, bor, dZor) = pycurious.tanaka1999(k, Phi, sigma_Phi, (0.11, 0.21), (0.11, 0.21))
    Zb, eZb = pycurious.ComputeTanaka(Ztr, dZtr, Zor, dZor)
    
    output[i] = [ws, Zb, eZb]
    #print('Zb estimate for '+str(ws)+' m: '+np.array2string(Zb)+', +/- '+np.array2string(eZb)+' km')

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6),)
ax1.plot(output[:,0], output[:,1])
ax1.set_ylim(0,30)
ax1.invert_yaxis()
ax1.set_xlabel('Data window size (m)')
ax1.set_ylabel('CPD estimate (km)')

ax2.plot(output[:,0], output[:,2])
ax2.set_xlabel('Data window size (m)')
ax2.set_ylabel('CPD misfit (km)')
#plt.gca().invert_yaxis()
#plt.xlabel('Data window size (km)')
#plt.ylabel('CPD depth (km)')

The results of this test highlight that there is a significant amount of variability in the CPD estimates at all window sizes, inferring that where possible, a range of window sizes should be tested for any data set. However, the righthand plot of propagated error in the CPD estimates clarifies that estimates from larger windows are more robust. In particular, with the prior knowledge that the CPD of the test data is 16 km, window sizes of greater than 80 km show a reduced improvement in statistics. 